This NB import processed URL data and extracts features from the article text. It then classifies the articles using a saved LSTM and exports a CSV of the results.

In [9]:
import pandas as pd
import numpy as np
import os
import operator 
import re
import sys

import seaborn as sns
from featurefunctions import create_text_column, features

##Feature Extraction

In [10]:
# import processed URL data

### article column MUST be labeled "text"
url_df = pd.read_csv("Final_Ayo_narratives_oct_2014.csv") 
url_df.head(3)

,Unnamed: 0,Topic_Num,Topic_Perc_Contrib,Keywords,Text_x,liststring,url,date,content,renderedContent,...,lang,source,sourceUrl,sourceLabel,media,retweetedTweet,quotedTweet,mentionedUsers,proc_text,Text_y
0,0,0.0,0.8609,"mexico, in, ejemplo, students, favor, pm, miss...","['``', 'Ayotzinapa', ',', 'punta', 'iceberg', ...","``,Ayotzinapa,,,punta,iceberg,fallido,'',,,opi...",https://twitter.com/AjedrezPolitiko/status/520...,2014-10-11 00:00:20+00:00,"""Ayotzinapa, la punta del iceberg de un estado...","""Ayotzinapa, la punta del iceberg de un estado...",...,es,"<a href=""http://www.hootsuite.com"" rel=""nofoll...",http://www.hootsuite.com,Hootsuite,NaN,NaN,NaN,"[{'username': 'BogusBelgodere', 'displayname':...","""Ayotzinapa, la punta del iceberg de un estado...","['``', 'Ayotzinapa', ',', 'punta', 'iceberg', ..."
1,1,1.0,0.8053,"zcalo, normalistas, padres, ayotzinapa, famili...","['Padres', 'normalistas', 'obligaron', 'devolv...","Padres,normalistas,obligaron,devolver,producto...",https://twitter.com/cb_television/status/52611...,2014-10-25 20:43:49+00:00,Padres de normalistas los obligaron a devolver...,Padres de normalistas los obligaron a devolver...,...,es,"<a href=""http://www.facebook.com/twitter"" rel=...",http://www.facebook.com/twitter,Facebook,NaN,NaN,NaN,NaN,Padres de normalistas los obligaron a devolver...,"['Padres', 'normalistas', 'obligaron', 'devolv..."
2,2,2.0,0.8053,"da, acto, familias, are, noche, gob, esto, hec...","['COPETE', ',', 'ZOQUETE', ',', 'Ayotzinapa', ...","COPETE,,,ZOQUETE,,,Ayotzinapa,PAULETTE,,,busca...",https://twitter.com/double_moral/status/520352...,2014-10-09 23:19:36+00:00,"COPETE, ZOQUETE, Ayotzinapa no es PAULETTE, bu...","COPETE, ZOQUETE, Ayotzinapa no es PAULETTE, bu...",...,es,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",http://twitter.com,Twitter Web Client,NaN,NaN,NaN,NaN,"COPETE, ZOQUETE, Ayotzinapa no es PAULETTE, bu...","['COPETE', ',', 'ZOQUETE', ',', 'Ayotzinapa', ..."


In [11]:
url_df= url_df.rename(columns = {'Text_y': 'Text'})  ##Turns column to Text so that I can match latertok_text 

In [12]:
# process text

text_df = create_text_column(url_df)
complete_df = text_df

features_df = complete_df.copy()
norm_df = complete_df.copy()

In [13]:
# add linguistic features

features_df = features(complete_df, features_df)


In [14]:
features_df.head()

,Unnamed: 0,Topic_Num,Topic_Perc_Contrib,Keywords,Text_x,liststring,url,date,content,renderedContent,...,Text,Sentiment,noun_count,verb_count,adj_count,adv_count,pron_count,char_count,word_count,word_density
0,0,0.0,0.8609,"mexico, in, ejemplo, students, favor, pm, miss...","['``', 'Ayotzinapa', ',', 'punta', 'iceberg', ...","``,Ayotzinapa,,,punta,iceberg,fallido,'',,,opi...",https://twitter.com/AjedrezPolitiko/status/520...,2014-10-11 00:00:20+00:00,"""Ayotzinapa, la punta del iceberg de un estado...","""Ayotzinapa, la punta del iceberg de un estado...",...,"['``', 'Ayotzinapa', ',', 'punta', 'iceberg', ...",0.497892,6.0,0.0,0.0,0.0,0.0,88.0,10.0,8.000000
1,1,1.0,0.8053,"zcalo, normalistas, padres, ayotzinapa, famili...","['Padres', 'normalistas', 'obligaron', 'devolv...","Padres,normalistas,obligaron,devolver,producto...",https://twitter.com/cb_television/status/52611...,2014-10-25 20:43:49+00:00,Padres de normalistas los obligaron a devolver...,Padres de normalistas los obligaron a devolver...,...,"['Padres', 'normalistas', 'obligaron', 'devolv...",0.416456,8.0,0.0,1.0,0.0,0.0,118.0,9.0,11.800000
2,2,2.0,0.8053,"da, acto, familias, are, noche, gob, esto, hec...","['COPETE', ',', 'ZOQUETE', ',', 'Ayotzinapa', ...","COPETE,,,ZOQUETE,,,Ayotzinapa,PAULETTE,,,busca...",https://twitter.com/double_moral/status/520352...,2014-10-09 23:19:36+00:00,"COPETE, ZOQUETE, Ayotzinapa no es PAULETTE, bu...","COPETE, ZOQUETE, Ayotzinapa no es PAULETTE, bu...",...,"['COPETE', ',', 'ZOQUETE', ',', 'Ayotzinapa', ...",0.233942,7.0,0.0,1.0,0.0,0.0,95.0,11.0,7.916667
3,3,3.0,0.9115,"prd, lo, dijo, ahora, ojos, ayotzinapa, imgene...","['Hay', 'prfugos', 'caso', 'normalistas', 'Ayo...","Hay,prfugos,caso,normalistas,Ayotzinapa,:,serv...",https://twitter.com/gii360/status/520330492248...,2014-10-09 21:50:31+00:00,Hay 4 prófugos por caso de normalistas de Ayot...,Hay 4 prófugos por caso de normalistas de Ayot...,...,"['Hay', 'prfugos', 'caso', 'normalistas', 'Ayo...",0.103857,10.0,0.0,1.0,0.0,0.0,131.0,12.0,10.076923
4,10,4.0,0.9026,"el, qu, ayotzinapa, por, caso, grupos, casa, u...","['Cronologa', 'Caso', 'Ayotzinapa', '-', 'El',...","Cronologa,Caso,Ayotzinapa,-,El,Universal,-,Ciu...",https://twitter.com/guerreroacapulc/status/525...,2014-10-24 00:15:00+00:00,Cronología Caso Ayotzinapa - El Universal - Ci...,Cronología Caso Ayotzinapa - El Universal - Ci...,...,"['Cronologa', 'Caso', 'Ayotzinapa', '-', 'El',...",0.123161,15.0,0.0,3.0,0.0,0.0,168.0,17.0,9.333333


In [15]:
features_df.to_csv('oct_2014.csv')